In [6]:
from ehc_sn.model import EIModel
import matplotlib.pyplot as plt
import torch

In [7]:
from ipywidgets import interact, IntSlider, FloatSlider
from functools import partial

IntSlider = partial(IntSlider, continuous_update=False)
FloatSlider = partial(FloatSlider, continuous_update=False)

In [8]:
def plot_raster(spikes, title):
    plt.figure(figsize=(10, 5))
    for neuron_idx in range(spikes.shape[1]):
        spike_times = torch.nonzero(spikes[:, neuron_idx]).squeeze()
        plt.scatter(spike_times, neuron_idx * torch.ones_like(spike_times), s=1)
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Neuron Index")
    plt.xlim(0, spikes.shape[0])
    plt.ylim(0, spikes.shape[1])
    plt.show()

In [9]:
model = EIModel(e_size=100, i_size=100)
model

EIModel(
  (excitatory): LIFRecurrentCell(input_size=100, hidden_size=100, p=LIFParameters(tau_syn_inv=tensor(200.), tau_mem_inv=tensor(100.), v_leak=tensor(0.), v_th=tensor(1.), v_reset=tensor(0.), method='super', alpha=tensor(100.)), autapses=False, dt=0.001)
  (inihibitory): LIFCell(p=LIFParameters(tau_syn_inv=tensor(200.), tau_mem_inv=tensor(100.), v_leak=tensor(0.), v_th=tensor(1.), v_reset=tensor(0.), method='super', alpha=tensor(100.)), dt=0.001)
)

In [ ]:
@interact(
    current=FloatSlider(min=0, max=10, step=0.1, value=2.4),
    t1=IntSlider(min=0, max=1000, step=1, value=200),
    t2=IntSlider(min=0, max=1000, step=1, value=1000),
)
def experiment(current, t1, t2):
    model.reset()
    input_currents = torch.zeros(1000, model.excitatory.input_size)
    input_currents[t1:t2, 0] = current  # Apply current to the first neuron
    outputs = model(input_currents)
    plot_raster(outputs, "Balance state spikes")

interactive(children=(FloatSlider(value=2.4, continuous_update=False, description='current', max=10.0), IntSli…